In [2]:
import pandas as pd
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import sklearn as sk
import pathlib as Path
import polars as pl
np.random.seed(42)

In [3]:
# Read the data from a CSV file from data/EstData.csv
data = pl.read_csv("../../data/EstData.csv")
data

ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64
1,58,0,0,0,18.6174,0,0,0,3,2
1,58,0,0,1,13.7783,0,0,0,3,2
1,58,0,0,2,16.5747,0,0,0,3,2
1,58,0,0,4,16.8486,0,0,0,3,2
1,58,0,0,8,18.6459,0,0,0,3,2
…,…,…,…,…,…,…,…,…,…,…
48,61,0,10,840,3.3777,0,0,0,3,2
48,61,0,10,1008,1.54816,0,0,0,2,1
48,61,0,10,1008,3.43845,0,0,0,3,2



### Overview of the data ###

Pharmacokinetics (PK): describes how the body interacts with the drug in terms of its absorption, distribution, metabolism, and elimination. It models how drug concentration changes over time in the body
Pharmacodynamics (PD): describes how the drug interacts with the body, linking drug concentration to its biological effect. PD models relate the concentration of a drug to the intensity and duration of its effects.

PK/PD: A combined PK/PD model integrates both aspects; it predicts the time course of drug effects based on the dosing regimen.

1.) ID: Subject Identifier

2.) BW: Body Weight (in kg)

3.) COMED:  Concomitant medication indicator for the subject (0: No, 1: Yes)

4.) DOSE: Dose level in mg. This column represents the amount of drug administered to the subject. It is typically measured in units like milligrams (mg) or micrograms (μg). The dose is a critical factor in determining the drug’s pharmacokinetic and pharmacodynamic properties.

5.) TIME: Time in hours. Indicates the time elapsed since the start of the first drug administration. 
                         Time is typically measured in hours or minutes and is essential for plotting concentration-time profiles.

6.) DV (Dependent Variable): Compound concentration (mg/L) for DVID=1. Biomarker level (ng/mL) for DVID=2. 
                    This column usually represents observed data, such as drug concentration in plasma or another biological matrix. 
                    It can also refer to the measurement of a biomarker or response variable affected by the drug, such as blood pressure or heart rate.

7.) EVID (Event ID): This is an event identifier used in NONMEM (a common software in pharmacometrics). It signifies the type of event occurring:
        EVID = 0 for observation events (e.g., a concentration measurement),
        EVID = 1 for dosing events
        
8.) MDV (Missing Dependent Variable): This value indicates whether the dependent variable (DV) is missing. An MDV of 1 means the DV value is missing, while 0 means it is present

9.) AMT (Amount): Stands for the actual dose amount administered, especially applicable during infusion dosing. AMT will be zero for observation records.

10.) CMT (Compartment): This denotes the compartment where the event (e.g., dosing or sampling) occurs. In PK models, different compartments (like central and peripheral) help to describe drug kinetics.

11.) DVID (Dependent Variable ID): This identifier helps distinguish between different types of DVs in the dataset. For example, you might have multiple measurement types such as concentrations and biomarkers.



In [5]:
def interpolate_nan_intervals_polars(df, column):
    """
    Replace intervals of null values in a Polars DataFrame column with linearly interpolated values
    from the first and last non-null values surrounding the interval.
    
    Parameters:
    -----------
    df : pl.DataFrame
        Input DataFrame
    column : str
        Name of the column to interpolate
        
    Returns:
    --------
    pl.DataFrame
        DataFrame with null intervals in the column replaced by linearly interpolated values
        
    Raises:
    -------
    ValueError
        If the first or last element of the column is null
    """
    # Check if first or last element is null
    if df[column].is_null()[0] or df[column].is_null()[-1]:
        raise ValueError("The first or last element of the column is null, cannot interpolate.")
    
    # Interpolate null values
    return df.with_columns(pl.col(column).interpolate())

In [6]:
data

ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64
1,58,0,0,0,18.6174,0,0,0,3,2
1,58,0,0,1,13.7783,0,0,0,3,2
1,58,0,0,2,16.5747,0,0,0,3,2
1,58,0,0,4,16.8486,0,0,0,3,2
1,58,0,0,8,18.6459,0,0,0,3,2
…,…,…,…,…,…,…,…,…,…,…
48,61,0,10,840,3.3777,0,0,0,3,2
48,61,0,10,1008,1.54816,0,0,0,2,1
48,61,0,10,1008,3.43845,0,0,0,3,2


In [7]:
# Find any nulls and replace with Polars interpolate function
# Not doing ID's since there are no Nans there
data = interpolate_nan_intervals_polars(data, "BW")
data = interpolate_nan_intervals_polars(data, "COMED")
data = interpolate_nan_intervals_polars(data, "DOSE")
data = interpolate_nan_intervals_polars(data, "TIME")
data = interpolate_nan_intervals_polars(data, "DV")

data = interpolate_nan_intervals_polars(data, "EVID")
data = interpolate_nan_intervals_polars(data, "MDV")
data = interpolate_nan_intervals_polars(data, "AMT")
data = interpolate_nan_intervals_polars(data, "CMT")
data = interpolate_nan_intervals_polars(data, "DVID")

data

ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,58.0,0.0,0.0,0.0,18.6174,0.0,0.0,0.0,3.0,2.0
1,58.0,0.0,0.0,1.0,13.7783,0.0,0.0,0.0,3.0,2.0
1,58.0,0.0,0.0,2.0,16.5747,0.0,0.0,0.0,3.0,2.0
1,58.0,0.0,0.0,4.0,16.8486,0.0,0.0,0.0,3.0,2.0
1,58.0,0.0,0.0,8.0,18.6459,0.0,0.0,0.0,3.0,2.0
…,…,…,…,…,…,…,…,…,…,…
48,61.0,0.0,10.0,840.0,3.3777,0.0,0.0,0.0,3.0,2.0
48,61.0,0.0,10.0,1008.0,1.54816,0.0,0.0,0.0,2.0,1.0
48,61.0,0.0,10.0,1008.0,3.43845,0.0,0.0,0.0,3.0,2.0


In [29]:
# Since data has now been preprocessed, Let's make PK Model

def pk_model(data):
    # Define the model structure
    model = {
        "BW": data["BW"].mean(),
        "COMED": data["COMED"].mean(),
        "DOSE": data["DOSE"].mean(),
        "TIME": data["TIME"].mean(),
        "DV": data["DV"].mean(),
        "EVID": data["EVID"].mean(),
        "MDV": data["MDV"].mean(),
        "AMT": data["AMT"].mean(),
        "CMT": data["CMT"].mean(),
        "DVID": data["DVID"].mean()
    }
    return model

print(pk_model(data))

{'BW': 73.08156028368795, 'COMED': 0.48404255319148937, 'DOSE': 4.170212765957447, 'TIME': 331.531914893617, 'DV': 3.4926242093404265, 'EVID': 0.2680851063829787, 'MDV': 0.2680851063829787, 'AMT': 1.251063829787234, 'CMT': 2.157446808510638, 'DVID': 1.1574468085106382}


In [8]:
'''Typical PK Model

--> State Variables (D(t) , A(t))
--> Input Variables (D)
--> Output Variables (C)
--> Parameters (variability)
--> Equations (differential)

PK Model should include at least:
1.) ID
2.) TIME
3.) DV (Dependent Variable: observed Value)
4.) AMT (Bolus amount (0 if none))
5.) MDV (Missing DV (also for dose record))
6.) CMT (compartment # of obs or dose (Larger models))
7.) DOSE (Dose amount)

'''

import scipy.integrate as solve_ivp
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

class PKModel:
    """
    Pharmacokinetic Model for population analysis; both one and two-compartment models with linear/nonlinear kinetics
    """
    
    def __init__(self, model_type='two_compartment'):
        """
        Initialize PK model
        
        Parameters:
        -----------
        model_type : str
            'one_compartment' or 'two_compartment'
        """
        self.model_type = model_type
        self.parameters = {}
        self.data = None
        
    def one_compartment_model(self, t, y, params):
        """
        One-compartment PK model with first-order elimination
        
        dy/dt = -k_el * y
        
        Parameters:
        -----------
        t : float
            Time
        y : array
            State variables [Central compartment amount]
        params : dict
            Model parameters {k_el, V}
        """
        A_central = y[0]
        k_el = params['k_el']
        
        dA_central_dt = -k_el * A_central
        
        return [dA_central_dt]
    
    def two_compartment_model(self, t, y, params):
        """
        Two-compartment PK model with distribution and elimination
        
        dA_central/dt = -k_el * A_central - k_12 * A_central + k_21 * A_peripheral
        dA_peripheral/dt = k_12 * A_central - k_21 * A_peripheral
        
        Parameters:
        -----------
        t : float
            Time
        y : array
            State variables [Central compartment amount, Peripheral compartment amount]
        params : dict
            Model parameters {k_el, k_12, k_21, V_central, V_peripheral}
        """
        A_central, A_peripheral = y
        k_el = params['k_el']
        k_12 = params['k_12']
        k_21 = params['k_21']
        
        dA_central_dt = -k_el * A_central - k_12 * A_central + k_21 * A_peripheral
        dA_peripheral_dt = k_12 * A_central - k_21 * A_peripheral
        
        return [dA_central_dt, dA_peripheral_dt]
    
    def simulate_concentration(self, time_points, dose, params, dosing_times=None):
        """
        Simulate drug concentration over time
        
        Parameters:
        -----------
        time_points : array
            Time points for simulation
        dose : float
            Dose amount
        params : dict
            Model parameters
        dosing_times : array, optional
            Times of additional doses
            
        Returns:
        --------
        array
            Simulated concentrations
        """
        if dosing_times is None:
            dosing_times = [0]
        
        # Initial conditions
        if self.model_type == 'one_compartment':
            y0 = [dose]  # Initial amount in central compartment
            model_func = self.one_compartment_model
        else:  # two_compartment
            y0 = [dose, 0]  # Initial amounts [central, peripheral]
            model_func = self.two_compartment_model
        
        # Solve ODE
        sol = solve_ivp.solve_ivp(
            lambda t, y: model_func(t, y, params),
            [time_points[0], time_points[-1]],
            y0,
            t_eval=time_points,
            method='RK45'
        )
        
        # Calculate concentration from central compartment
        if sol.success:
            central_amounts = sol.y[0]
            concentrations = central_amounts / params['V_central']
            return concentrations
        else:
            return np.zeros_like(time_points)
    
    def population_parameters(self, df):
        """
        Estimate population pharmacokinetic parameters
        
        Parameters:
        -----------
        df : pl.DataFrame
            Dataset with PK data
            
        Returns:
        --------
        dict
            Population parameter estimates and variability
        """
        # Separate concentration data (DVID=1) and biomarker data (DVID=2)
        conc_data = df.filter(pl.col("DVID") == 1)
        biomarker_data = df.filter(pl.col("DVID") == 2)
        
        # Group by subject
        subjects = df.select("ID").unique().sort("ID")
        
        individual_params = []
        
        for subject_id in subjects.to_series():
            subject_data = conc_data.filter(pl.col("ID") == subject_id)
            
            if len(subject_data) > 0:
                # Extract relevant data
                times = subject_data.select("TIME").to_series().to_numpy()
                concentrations = subject_data.select("DV").to_series().to_numpy()
                doses = subject_data.select("DOSE").to_series().to_numpy()
                
                # Get dose information (assuming first dose is representative)
                dose = doses[doses > 0][0] if any(doses > 0) else 10  # Default dose
                
                # Estimate individual parameters
                params = self._fit_individual_params(times, concentrations, dose)
                if params:
                    individual_params.append(params)
        
        # Calculate population statistics
        if individual_params:
            pop_params = self._calculate_population_stats(individual_params)
            return pop_params
        else:
            return self._default_parameters()
    
    def _fit_individual_params(self, times, concentrations, dose):
        """
        Fit PK parameters for individual subject
        """
        def objective(params_array):
            if self.model_type == 'one_compartment':
                params = {
                    'k_el': abs(params_array[0]),
                    'V_central': abs(params_array[1])
                }
            else:  # two_compartment
                params = {
                    'k_el': abs(params_array[0]),
                    'k_12': abs(params_array[1]),
                    'k_21': abs(params_array[2]),
                    'V_central': abs(params_array[3]),
                    'V_peripheral': abs(params_array[4])
                }
            
            try:
                pred_conc = self.simulate_concentration(times, dose, params)
                # Remove zero concentrations for log-likelihood
                valid_idx = (concentrations > 0) & (pred_conc > 0)
                if np.sum(valid_idx) < 2:
                    return 1e6
                
                # Weighted least squares
                residuals = np.log(concentrations[valid_idx]) - np.log(pred_conc[valid_idx])
                return np.sum(residuals**2)
            except:
                return 1e6
        
        # Initial parameter estimates
        if self.model_type == 'one_compartment':
            initial_params = [0.1, 10.0]  # k_el, V_central
            bounds = [(0.001, 1.0), (1.0, 100.0)]
        else:  # two_compartment
            initial_params = [0.1, 0.05, 0.02, 10.0, 20.0]  # k_el, k_12, k_21, V_central, V_peripheral
            bounds = [(0.001, 1.0), (0.001, 0.5), (0.001, 0.5), (1.0, 100.0), (1.0, 200.0)]
        
        try:
            result = minimize(objective, initial_params, method='L-BFGS-B', bounds=bounds)
            if result.success:
                if self.model_type == 'one_compartment':
                    return {
                        'k_el': abs(result.x[0]),
                        'V_central': abs(result.x[1]),
                        'CL': abs(result.x[0] * result.x[1]),  # Clearance
                        't_half': np.log(2) / abs(result.x[0])  # Half-life
                    }
                else:
                    return {
                        'k_el': abs(result.x[0]),
                        'k_12': abs(result.x[1]),
                        'k_21': abs(result.x[2]),
                        'V_central': abs(result.x[3]),
                        'V_peripheral': abs(result.x[4]),
                        'CL': abs(result.x[0] * result.x[3]),
                        'Q': abs(result.x[1] * result.x[3]),  # Intercompartmental clearance
                        'V_ss': abs(result.x[3]) + abs(result.x[4])  # Volume at steady state
                    }
        except:
            pass
        
        return None
    
    def _calculate_population_stats(self, individual_params):
        """
        Calculate population parameter statistics
        """
        pop_stats = {}
        
        # Get all parameter names
        param_names = individual_params[0].keys()
        
        for param in param_names:
            values = [p[param] for p in individual_params if param in p]
            if values:
                pop_stats[param] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'cv': np.std(values) / np.mean(values) * 100,  # Coefficient of variation
                    'median': np.median(values),
                    'range': (np.min(values), np.max(values))
                }
        
        return pop_stats
    
    def _default_parameters(self):
        """
        Return default parameters if fitting fails
        """
        if self.model_type == 'one_compartment':
            return {
                'k_el': {'mean': 0.1, 'std': 0.02, 'cv': 20},
                'V_central': {'mean': 10.0, 'std': 2.0, 'cv': 20},
                'CL': {'mean': 1.0, 'std': 0.2, 'cv': 20},
                't_half': {'mean': 6.93, 'std': 1.39, 'cv': 20}
            }
        else:
            return {
                'k_el': {'mean': 0.1, 'std': 0.02, 'cv': 20},
                'k_12': {'mean': 0.05, 'std': 0.01, 'cv': 20},
                'k_21': {'mean': 0.02, 'std': 0.005, 'cv': 25},
                'V_central': {'mean': 10.0, 'std': 2.0, 'cv': 20},
                'V_peripheral': {'mean': 20.0, 'std': 5.0, 'cv': 25},
                'CL': {'mean': 1.0, 'std': 0.2, 'cv': 20},
                'Q': {'mean': 0.5, 'std': 0.1, 'cv': 20},
                'V_ss': {'mean': 30.0, 'std': 7.0, 'cv': 23}
            }

# Initialize and run PK analysis
print("Running PK model...")

# One-compartment model analysis
pk_one = PKModel('one_compartment')
pop_params_one = pk_one.population_parameters(data)

print("\n=== ONE-COMPARTMENT MODEL RESULTS ===")
for param, stats in pop_params_one.items():
    print(f"{param}:")
    print(f"  Mean: {stats['mean']:.4f}")
    print(f"  CV%: {stats['cv']:.1f}%")
    print(f"  Range: {stats['range'][0]:.4f} - {stats['range'][1]:.4f}")

# Two-compartment model analysis
pk_two = PKModel('two_compartment')
pop_params_two = pk_two.population_parameters(data)

print("\n=== TWO-COMPARTMENT MODEL RESULTS ===")
for param, stats in pop_params_two.items():
    print(f"{param}:")
    print(f"  Mean: {stats['mean']:.4f}")
    print(f"  CV%: {stats['cv']:.1f}%")
    print(f"  Range: {stats['range'][0]:.4f} - {stats['range'][1]:.4f}")

Running PK model...

=== ONE-COMPARTMENT MODEL RESULTS ===
k_el:
  Mean: 0.0011
  CV%: 19.4%
  Range: 0.0010 - 0.0019
V_central:
  Mean: 2.1705
  CV%: 31.8%
  Range: 1.1305 - 3.4527
CL:
  Mean: 0.0024
  CV%: 48.8%
  Range: 0.0011 - 0.0067
t_half:
  Mean: 652.7283
  CV%: 13.5%
  Range: 358.0925 - 693.1472

=== ONE-COMPARTMENT MODEL RESULTS ===
k_el:
  Mean: 0.0011
  CV%: 19.4%
  Range: 0.0010 - 0.0019
V_central:
  Mean: 2.1705
  CV%: 31.8%
  Range: 1.1305 - 3.4527
CL:
  Mean: 0.0024
  CV%: 48.8%
  Range: 0.0011 - 0.0067
t_half:
  Mean: 652.7283
  CV%: 13.5%
  Range: 358.0925 - 693.1472

=== TWO-COMPARTMENT MODEL RESULTS ===
k_el:
  Mean: 0.2579
  CV%: 169.3%
  Range: 0.0010 - 1.0000
k_12:
  Mean: 0.2414
  CV%: 83.4%
  Range: 0.0010 - 0.5000
k_21:
  Mean: 0.4151
  CV%: 39.3%
  Range: 0.0010 - 0.5000
V_central:
  Mean: 1.7455
  CV%: 49.3%
  Range: 1.0000 - 3.7620
V_peripheral:
  Mean: 20.0000
  CV%: 0.0%
  Range: 20.0000 - 20.0000
CL:
  Mean: 0.2587
  CV%: 168.5%
  Range: 0.0010 - 1.0000


In [ ]:
# PK Variability Analysis with Visualization

def covariate_analysis(df):
    """
    Covariate analysis for PK parameters
    """
    print("\n=== COVARIATE ANALYSIS ===")
    
    conc_data = df.filter(pl.col("DVID") == 1)
    
    # Calculate PK metrics for each subject
    pk_metrics = conc_data.group_by("ID").agg([
        pl.col("DV").sum().alias("auc_approx"),
        pl.col("DV").max().alias("cmax"),
        pl.col("DV").mean().alias("cavg"),
        pl.col("DOSE").max().alias("dose"),
        pl.col("BW").first().alias("bw"),
        pl.col("COMED").first().alias("comed"),
        pl.col("TIME").max().alias("study_duration"),
        pl.count().alias("n_observations")
    ]).with_columns([
        (pl.col("dose") / pl.col("auc_approx")).alias("apparent_cl"),
        (pl.col("dose") / pl.col("cmax")).alias("apparent_v"),
        (pl.col("auc_approx") / pl.col("study_duration")).alias("exposure_rate")
    ]).filter(pl.col("apparent_cl").is_finite() & pl.col("apparent_v").is_finite())
    
    print("Enhanced PK Metrics Summary:")
    print(pk_metrics.select(['apparent_cl', 'apparent_v', 'cmax', 'bw', 'dose']).describe())
    
    # Advanced correlation analysis
    correlations = {}
    metrics = ['apparent_cl', 'apparent_v', 'cmax', 'auc_approx']
    covariates = ['bw', 'dose']
    
    for metric in metrics:
        correlations[metric] = {}
        for cov in covariates:
            r = np.corrcoef(pk_metrics.select(metric).to_series(), 
                           pk_metrics.select(cov).to_series())[0,1]
            correlations[metric][cov] = r
    
    print("\n=== CORRELATION MATRIX ===")
    print("Metric\\Covariate\tBW\tDOSE")
    for metric in metrics:
        print(f"{metric:<15}\t{correlations[metric]['bw']:.3f}\t{correlations[metric]['dose']:.3f}")
    
    # Statistical significance of concomitant medication
    no_comed = pk_metrics.filter(pl.col("comed") == 0)
    with_comed = pk_metrics.filter(pl.col("comed") == 1)
    
    print(f"\n=== CONCOMITANT MEDICATION COMPARISON ===")
    print(f"No COMED (n={len(no_comed)}):")
    print(f"  Apparent CL: {no_comed.select('apparent_cl').mean().item():.4f} ± {no_comed.select('apparent_cl').std().item():.4f}")
    print(f"  Apparent V:  {no_comed.select('apparent_v').mean().item():.4f} ± {no_comed.select('apparent_v').std().item():.4f}")
    print(f"  Cmax:       {no_comed.select('cmax').mean().item():.4f} ± {no_comed.select('cmax').std().item():.4f}")
    
    print(f"With COMED (n={len(with_comed)}):")
    print(f"  Apparent CL: {with_comed.select('apparent_cl').mean().item():.4f} ± {with_comed.select('apparent_cl').std().item():.4f}")
    print(f"  Apparent V:  {with_comed.select('apparent_v').mean().item():.4f} ± {with_comed.select('apparent_v').std().item():.4f}")
    print(f"  Cmax:       {with_comed.select('cmax').mean().item():.4f} ± {with_comed.select('cmax').std().item():.4f}")
    
    # Dose linearity assessment
    dose_linearity = pk_metrics.group_by("dose").agg([
        pl.col("auc_approx").mean().alias("mean_auc"),
        pl.col("auc_approx").std().alias("std_auc"),
        pl.col("cmax").mean().alias("mean_cmax"),
        pl.col("cmax").std().alias("std_cmax"),
        pl.count().alias("n_subjects")
    ]).with_columns([
        (pl.col("mean_auc") / pl.col("dose")).alias("auc_dose_ratio"),
        (pl.col("mean_cmax") / pl.col("dose")).alias("cmax_dose_ratio")
    ])
    
    print(f"\n=== DOSE LINEARITY ASSESSMENT ===")
    print(dose_linearity)
    
    return pk_metrics, correlations

def create_final_pk_visualization(df, pk_metrics):
    """
    Create PK visualization
    """
    plt.figure(figsize=(16, 12))
    
    # Plot 1: Clearance vs Body Weight by Dose
    plt.subplot(3, 3, 1)
    doses = pk_metrics.select("dose").unique().sort("dose").to_series()
    colors = ['purple', 'teal', 'gold']
    
    for i, dose in enumerate(doses):
        dose_data = pk_metrics.filter(pl.col("dose") == dose)
        plt.scatter(dose_data.select("bw").to_series(), 
                   dose_data.select("apparent_cl").to_series(),
                   c=colors[i], label=f'{dose} mg', alpha=0.7, s=50)
    
    plt.xlabel('Body Weight (kg)')
    plt.ylabel('Apparent Clearance')
    plt.title('Clearance vs Body Weight')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Volume vs Body Weight
    plt.subplot(3, 3, 2)
    for i, dose in enumerate(doses):
        dose_data = pk_metrics.filter(pl.col("dose") == dose)
        plt.scatter(dose_data.select("bw").to_series(), 
                   dose_data.select("apparent_v").to_series(),
                   c=colors[i], label=f'{dose} mg', alpha=0.7, s=50)
    
    plt.xlabel('Body Weight (kg)')
    plt.ylabel('Apparent Volume')
    plt.title('Volume vs Body Weight')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Parameter CV% comparison
    plt.subplot(3, 3, 3)
    cv_one = [48.8, 31.8, 13.5]  # CL, V, t_half for one-compartment
    cv_two = [168.5, 49.3, 20.0]  # Approximate values for two-compartment
    
    params = ['CL', 'V', 't½']
    x = np.arange(len(params))
    width = 0.35
    
    plt.bar(x - width/2, cv_one, width, label='One-compartment', alpha=0.7, color='skyblue')
    plt.bar(x + width/2, cv_two, width, label='Two-compartment', alpha=0.7, color='lightcoral')
    
    plt.xlabel('PK Parameters')
    plt.ylabel('CV (%)')
    plt.title('Parameter Variability')
    plt.xticks(x, params)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 4: Dose-normalized AUC
    plt.subplot(3, 3, 4)
    dose_norm_auc = pk_metrics.with_columns(
        (pl.col("auc_approx") / pl.col("dose")).alias("dose_norm_auc")
    )
    
    for i, dose in enumerate(doses):
        dose_data = dose_norm_auc.filter(pl.col("dose") == dose)
        plt.scatter(dose_data.select("bw").to_series(), 
                   dose_data.select("dose_norm_auc").to_series(),
                   c=colors[i], label=f'{dose} mg', alpha=0.7, s=50)
    
    plt.xlabel('Body Weight (kg)')
    plt.ylabel('Dose-normalized AUC')
    plt.title('Dose-normalized Exposure')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 5: Concomitant medication effects
    plt.subplot(3, 3, 5)
    no_comed = pk_metrics.filter(pl.col("comed") == 0)
    with_comed = pk_metrics.filter(pl.col("comed") == 1)
    
    plt.boxplot([no_comed.select("apparent_cl").to_series().to_list(),
                 with_comed.select("apparent_cl").to_series().to_list()],
                labels=['No COMED', 'With COMED'])
    plt.ylabel('Apparent Clearance')
    plt.title('COMED Effect on Clearance')
    plt.grid(True, alpha=0.3)
    
    # Plot 6: Individual profiles (subset)
    plt.subplot(3, 3, 6)
    conc_data = df.filter(pl.col("DVID") == 1)
    subjects = conc_data.select("ID").unique().sort("ID").head(8)
    
    for subject_id in subjects.to_series():
        subject_data = conc_data.filter(pl.col("ID") == subject_id)
        times = subject_data.select("TIME").to_series()
        concentrations = subject_data.select("DV").to_series()
        plt.semilogy(times, concentrations, 'o-', alpha=0.6, markersize=3)
    
    plt.xlabel('Time (hours)')
    plt.ylabel('Concentration (mg/L)')
    plt.title('Individual PK Profiles (Log Scale)')
    plt.grid(True, alpha=0.3)
    
    # Plot 7-9: Distribution histograms
    plt.subplot(3, 3, 7)
    plt.hist(pk_metrics.select("apparent_cl").to_series(), bins=15, alpha=0.7, color='skyblue')
    plt.xlabel('Apparent Clearance')
    plt.ylabel('Frequency')
    plt.title('Clearance Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 3, 8)
    plt.hist(pk_metrics.select("apparent_v").to_series(), bins=15, alpha=0.7, color='lightgreen')
    plt.xlabel('Apparent Volume')
    plt.ylabel('Frequency')
    plt.title('Volume Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 3, 9)
    plt.hist(pk_metrics.select("cmax").to_series(), bins=15, alpha=0.7, color='orange')
    plt.xlabel('Cmax (mg/L)')
    plt.ylabel('Frequency')
    plt.title('Cmax Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("=== PK MODEL ASSESSMENT ===")

# Covariate analysis
pk_metrics, correlations = covariate_analysis(data)

# Create final visualization
create_final_pk_visualization(data, pk_metrics)


print(f"Dataset: {len(data)} observations from {data.select('ID').n_unique()} subjects")
print(f"Concentration data: {len(data.filter(pl.col('DVID') == 1))} observations")
print(f"Biomarker data: {len(data.filter(pl.col('DVID') == 2))} observations")

=== PK MODEL ASSESSMENT ===

=== COVARIATE ANALYSIS ===
Enhanced PK Metrics Summary:
shape: (9, 6)
┌────────────┬─────────────┬────────────┬──────────┬───────────┬──────────┐
│ statistic  ┆ apparent_cl ┆ apparent_v ┆ cmax     ┆ bw        ┆ dose     │
│ ---        ┆ ---         ┆ ---        ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ f64         ┆ f64        ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════╪══════════╪═══════════╪══════════╡
│ count      ┆ 36.0        ┆ 36.0       ┆ 36.0     ┆ 36.0      ┆ 36.0     │
│ null_count ┆ 0.0         ┆ 0.0        ┆ 0.0      ┆ 0.0       ┆ 0.0      │
│ mean       ┆ 0.098057    ┆ 1.018699   ┆ 4.86097  ┆ 72.694444 ┆ 4.666667 │
│ std        ┆ 0.034053    ┆ 0.353596   ┆ 4.049432 ┆ 16.049601 ┆ 3.913347 │
│ min        ┆ 0.052129    ┆ 0.542791   ┆ 0.563261 ┆ 51.0      ┆ 1.0      │
│ 25%        ┆ 0.068803    ┆ 0.742528   ┆ 1.2629   ┆ 58.0      ┆ 1.0      │
│ 50%        ┆ 0.094791    ┆ 0.982495   ┆ 4.41675  ┆ 70.0      ┆ 